<Strong> Fixed size time-series which we slide in time to predict into future. </Strong>

In [136]:
import pandas as pd
import numpy as np
from keras.models import Sequential 
from keras import layers
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Activation, Dense

In [137]:
df = pd.read_csv("../../../cryptoData/BTC_15m_data.csv")

In [138]:
df

,date,open,high,low,close,volume
0,2017-08-17 04:00:00,4261.48,4280.56,4261.48,4261.48,2.189061
1,2017-08-17 04:15:00,4261.48,4270.41,4261.32,4261.45,9.119865
2,2017-08-17 04:30:00,4280.00,4310.07,4267.99,4310.07,21.923552
3,2017-08-17 04:45:00,4310.07,4313.62,4291.37,4308.83,13.948531
4,2017-08-17 05:00:00,4308.83,4328.69,4304.31,4304.31,5.101153
...,...,...,...,...,...,...
155486,2022-01-28 14:45:00,36684.11,36828.38,36600.00,36759.65,560.007760
155487,2022-01-28 15:00:00,36759.65,37198.00,36682.24,37149.99,649.317780
155488,2022-01-28 15:15:00,37150.00,37300.00,36925.89,36988.91,1061.113920
155489,2022-01-28 15:30:00,36985.43,37115.32,36874.46,37022.42,452.420950


In [139]:
float_data = df.values[:, 1: ,].astype(float)
print(float_data)

[[4.26148000e+03 4.28056000e+03 4.26148000e+03 4.26148000e+03
  2.18906100e+00]
 [4.26148000e+03 4.27041000e+03 4.26132000e+03 4.26145000e+03
  9.11986500e+00]
 [4.28000000e+03 4.31007000e+03 4.26799000e+03 4.31007000e+03
  2.19235520e+01]
 ...
 [3.71500000e+04 3.73000000e+04 3.69258900e+04 3.69889100e+04
  1.06111392e+03]
 [3.69854300e+04 3.71153200e+04 3.68744600e+04 3.70224200e+04
  4.52420950e+02]
 [3.70238600e+04 3.72715400e+04 3.70003400e+04 3.72310200e+04
  2.54796250e+02]]


In [140]:
data = float_data[:,3,]
print(data)

[ 4261.48  4261.45  4310.07 ... 36988.91 37022.42 37231.02]


In [135]:
'''
def rolling_window(train_data , window_size , predict_future_steps):
    while True:
    
        for i in range (len(train_data) - window_size - predict_future_steps + 1):

            split_train = train_data[i:window_size+i]
            split_val = train_data[i+window_size:window_size+i+predict_future_steps]

            yield split_train , split_val
            
rolling = rolling_window(subData , 10 , 1)

for train , val in rolling:
    print(train.shape)
    print(val.shape)
    break
    
'''

'\ndef rolling_window(train_data , window_size , predict_future_steps):\n    while True:\n    \n        for i in range (len(train_data) - window_size - predict_future_steps + 1):\n\n            split_train = train_data[i:window_size+i]\n            split_val = train_data[i+window_size:window_size+i+predict_future_steps]\n\n            yield split_train , split_val\n'

In [144]:
subData = data[:100]

In [141]:
fh = np.arange(1, len(subData) + 1)  # we add 1 because the `stop` value is exclusive in `np.arange`
fh

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [145]:

# slightly modified code from the M4 competition
def split_into_train_test(data, in_num, fh):
    """
    Splits the series into train and test sets. Each step takes multiple points as inputs
    :param data: an individual TS
    :param fh: number of out of sample points
    :param in_num: number of input points for the forecast
    :return:
    """
    train, test = data[:-fh], data[-(fh + in_num):]
    x_train, y_train = train[:-1], np.roll(train, -in_num)[:-in_num]
    x_test, y_test = test[:-1], np.roll(test, -in_num)[:-in_num]
#     x_test, y_test = train[-in_num:], np.roll(test, -in_num)[:-in_num]

    # reshape input to be [samples, time steps, features] (N-NF samples, 1 time step, 1 feature)
    x_train = np.reshape(x_train, (-1, 1))
    x_test = np.reshape(x_test, (-1, 1))
    temp_test = np.roll(x_test, -1)
    temp_train = np.roll(x_train, -1)
    for x in range(1, in_num):
        x_train = np.concatenate((x_train[:-1], temp_train[:-1]), 1)
        x_test = np.concatenate((x_test[:-1], temp_test[:-1]), 1)
        temp_test = np.roll(temp_test, -1)[:-1]
        temp_train = np.roll(temp_train, -1)[:-1]

    return x_train, y_train, x_test, y_test

In [152]:
# now we can split the actual values of the time series
x_train, y_train, x_test, y_test = split_into_train_test(subData, 10, 1)

In [164]:
print(x_train.shape)

(89, 10)


In [154]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

In [155]:
model.compile(optimizer=RMSprop(), loss='mae')

In [156]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 12)                132       
                                                                 
 dense_29 (Dense)            (None, 8)                 104       
                                                                 
 dense_30 (Dense)            (None, 1)                 9         
                                                                 
Total params: 245
Trainable params: 245
Non-trainable params: 0
_________________________________________________________________


In [157]:
model.fit(x_train, y_train)

3/3 [==============================] - 1s 6ms/step - loss: 2159.1406
